# Figure 2
- Py Kernel with rpy2 integration for R and Python

In [1]:
import pandas as pd
import sys
sys.path.insert(1, '/Users/brinkley97/Documents/development/')
import utils

# Load Data

In [2]:
path_to_data =  "lab-kcad/datasets/tiles_dataset/" 
work_data = "figure_2/physical/diurnal_work_lm_6.csv.gz"
off_data = "figure_2/physical/diurnal_off_lm_6.csv.gz"

In [3]:
work_df = utils.load_gzip_csv_data(path_to_data, work_data)
off_df = utils.load_gzip_csv_data(path_to_data, off_data)

In [4]:
# work_df

In [5]:
# off_df

# Modify Dataframes

In [6]:
def modify_df(work_df, off_df):
    '''Modify Dataframes columns and values to match/flow with specific questions
    
    Arguments:
    work_df -- Dataframe
    off_df -- Dataframe
    
    Return: tuple/list 
    work_df -- Dataframe
    off_df -- Dataframe
    '''
    # 0 is actually 11pm - 3am, ..., 5 is 7pm - 11pm
    # day starts with 2, which is 7am -11am, night starts with 5, which is 7pm - 11pm
    day_map = {'3rd': "11:00–15:00 h", '4th': "15:00–19:00 h", '5th': "19:00–23:00 h", '6th': "23:00–3:00 h", '1st': "3:00–7:00 h", '2nd': "7:00–11:00 h"}
    night_map = {'6th': '23:00–3:00 h', '1st': '3:00–7:00 h', '2nd': '7:00–11:00 h', '3rd': '11:00–15:00 h', '4th': '15:00–19:00 h', '5th': '19:00–23:00 h'}
    shift_map = {'day': 'day-shift', 'night': 'night-shift'}
    
    work_df_copy = work_df.copy()
    # work_df_copy = work_df_copy.drop(columns=["age",  "gender"])
    update_work_df = work_df_copy.replace({"time": day_map})
    update_work_df = update_work_df.replace({"shift": shift_map})
    
    off_df_copy = off_df.copy()
    update_off_df = off_df_copy.replace({"time": night_map})
    update_off_df = update_off_df.replace({"shift": shift_map})
                            
    return update_work_df, update_off_df

In [7]:
modified_work_df, modified_off_df = modify_df(work_df, off_df)

In [8]:
# modified_work_df

# Load Generated Specific Questions

In [9]:
base = "/Users/brinkley97/Documents/development/lab-kcad/"
path_to_questions = "tiles-day-night/my_code"
name_of_questions = "generateSpecificQuestions.ipynb"
questions = base + path_to_questions + name_of_questions

In [10]:
# %load questions
/Users/brinkley97/Documents/development/lab-kcad/tiles-day-night/my_codegenerateSpecificQuestions.ipynb

In [11]:
%run "generateSpecificQuestions.ipynb"

In [12]:
# fig_2

# Py + R Code Integration

In [13]:
# %load_ext rpy2.ipython

In [14]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages

# load and activate() bc if NOT error (Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>') will appear
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [15]:
'''
Install packages from the first mirror (ind=1) known to R is done with
'''
# utils = rpackages.importr('utils')

# # select a mirror for R packages
# utils.chooseCRANmirror(ind=1)

'\nInstall packages from the first mirror (ind=1) known to R is done with\n'

In [16]:
# R package names
# packnames = ('devtools')

# R vector of strings
# from rpy2.robjects.vectors import StrVector

# # Selectively install what needs to be install.
# names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
# if len(names_to_install) > 0:
#     utils.install_packages(StrVector(names_to_install))

In [17]:
report = rpackages.importr('report')

SyntaxError: invalid syntax (3122380276.py, line 2)

In [53]:
rpackages.importr('devtools')
rpackages.importr('ez')

SyntaxError: invalid syntax (1337385385.py, line 2)

In [19]:
# see all cols; run here, then go back to cell w/ data to run
pd.set_option('display.max_columns', 9)

In [20]:
# def separate_data(df, specific_question):
    

In [62]:
def figure_2(generated_specific_questions, work_df, off_df):
    '''Reproduce Table 2
    
    Arguments:
    generated_specific_questions -- list
    work_df -- Dataframe
    off_df -- Dataframe
    
    Return: specific_data -- Dataframe in list
    
    '''
    # print(type(generated_specific_questions), generated_specific_questions)
    # print(type(work_df), work_df)
    # print(type(off_df), off_df)
    
    work_data_columns = list(work_df.keys())
    print(work_data_columns)
   
    times = ['23:00–3:00 h', '3:00–7:00 h', '7:00–11:00 h', '11:00–15:00 h', '15:00–19:00 h', '19:00–23:00 h']
   
    for time in times:
        work_data_columns.append(time)
    
    for generated_specific_question in range(len(generated_specific_questions)):
        specific_question = generated_specific_questions[generated_specific_question]
        print("specific_question : ", specific_question)
        
        data_cols = ["time", "id", "shift"]
        time = []
        
        for work_data_column in work_data_columns: 
            
            if work_data_column in specific_question.split("*"):
                # print(work_data_column, True)
                data_cols.append(work_data_column)
                time.append(work_data_column)
                
                
            else:
                # print(work_data_column, False)
                continue
                
            
        # print(generated_specific_question, data_cols[0:-1], time[-1])
        
        data_cols = data_cols[0:-1]
        set_time = time[-1]
        form_df = work_df.loc[0:, data_cols]
        
        
        specific_work_df = form_df.set_index("time")
        # print(specific_work_df, "\n-----")
        
#         specific_work_df = specific_work_df.loc[set_time]
        # print(specific_work_df, "\n-----")
        
        if len(specific_work_df.keys()) == 4:
#             print(specific_work_df)
            
            update_work_df = specific_work_df.reset_index()
#             print(update_work_df)
            
            '''
            Start of integration
            '''
            
            r_objects = robjects.r
            r_objects.source("figure_2.R")
            print("\n=========================")
            work_time_shift_id = r_objects.work_model(update_work_df)
            print(work_time_shift_id)

In [63]:
figure_2(fig_2, modified_work_df, modified_off_df)

['Unnamed: 0', 'work', 'time', 'rest', 'step_ratio', 'run_ratio', 'id', 'age', 'gender', 'shift']
specific_question :  on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *mid* between the hours of *23:00–3:00 h* ?
specific_question :  on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *mid* between the hours of *3:00–7:00 h* ?
specific_question :  on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *mid* between the hours of *7:00–11:00 h* ?
specific_question :  on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *mid* between the hours of *11:00–15:00 h* ?
specific_question :  on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *mid* 

R[write to console]: Warning:
R[write to console]:  Converting "id" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "time" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "shift" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA().



R[write to console]: Warning:
R[write to console]:  Converting "id" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "time" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "shift" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA().




  Posthoc multiple comparisons of means : Fisher LSD 
    95% family-wise confidence level

$shift
                            diff     lwr.ci    upr.ci    pval    
night-shift-day-shift 0.07810084 0.05280674 0.1033949 2.3e-09 ***

$time
                                    diff       lwr.ci       upr.ci    pval    
15:00–19:00 h-11:00–15:00 h -0.001001384 -0.043780869  0.041778101  0.9633    
19:00–23:00 h-11:00–15:00 h -0.051026143 -0.093805628 -0.008246657  0.0195 *  
23:00–3:00 h-11:00–15:00 h  -0.013750695 -0.056530180  0.029028791  0.5281    
3:00–7:00 h-11:00–15:00 h    0.138635316  0.095855831  0.181414801 3.9e-10 ***
7:00–11:00 h-11:00–15:00 h   0.123941858  0.081162373  0.166721343 2.0e-08 ***
19:00–23:00 h-15:00–19:00 h -0.050024758 -0.092804244 -0.007245273  0.0220 *  
23:00–3:00 h-15:00–19:00 h  -0.012749310 -0.055528796  0.030030175  0.5586    
3:00–7:00 h-15:00–19:00 h    0.139636700  0.096857215  0.182416185 2.9e-10 ***
7:00–11:00 h-15:00–19:00 h   0.124943242  0.082163

R[write to console]: Warning:
R[write to console]:  Converting "id" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "time" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "shift" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA().



...

R[write to console]: Warning:
R[write to console]:  Converting "id" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "time" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "shift" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Data is unbalanced (unequal N per group). Make sure you specified a well-considered value for the type argument to ezANOVA().




  Posthoc multiple comparisons of means : Fisher LSD 
    95% family-wise confidence level

$shift
                            diff     lwr.ci    upr.ci    pval    
night-shift-day-shift 0.07810084 0.05280674 0.1033949 2.3e-09 ***

$time
                                    diff       lwr.ci       upr.ci    pval    
15:00–19:00 h-11:00–15:00 h -0.001001384 -0.043780869  0.041778101  0.9633    
19:00–23:00 h-11:00–15:00 h -0.051026143 -0.093805628 -0.008246657  0.0195 *  
23:00–3:00 h-11:00–15:00 h  -0.013750695 -0.056530180  0.029028791  0.5281    
3:00–7:00 h-11:00–15:00 h    0.138635316  0.095855831  0.181414801 3.9e-10 ***
7:00–11:00 h-11:00–15:00 h   0.123941858  0.081162373  0.166721343 2.0e-08 ***
19:00–23:00 h-15:00–19:00 h -0.050024758 -0.092804244 -0.007245273  0.0220 *  
23:00–3:00 h-15:00–19:00 h  -0.012749310 -0.055528796  0.030030175  0.5586    
3:00–7:00 h-15:00–19:00 h    0.139636700  0.096857215  0.182416185 2.9e-10 ***
7:00–11:00 h-15:00–19:00 h   0.124943242  0.082163

R[write to console]: Error in ezANOVA_main(data = data, dv = dv, wid = wid, within = within,  : 
  "rest" is not a variable in the data frame provided.




  Posthoc multiple comparisons of means : Fisher LSD 
    95% family-wise confidence level

$shift
                            diff     lwr.ci    upr.ci    pval    
night-shift-day-shift 0.07810084 0.05280674 0.1033949 2.3e-09 ***

$time
                                    diff       lwr.ci       upr.ci    pval    
15:00–19:00 h-11:00–15:00 h -0.001001384 -0.043780869  0.041778101  0.9633    
19:00–23:00 h-11:00–15:00 h -0.051026143 -0.093805628 -0.008246657  0.0195 *  
23:00–3:00 h-11:00–15:00 h  -0.013750695 -0.056530180  0.029028791  0.5281    
3:00–7:00 h-11:00–15:00 h    0.138635316  0.095855831  0.181414801 3.9e-10 ***
7:00–11:00 h-11:00–15:00 h   0.123941858  0.081162373  0.166721343 2.0e-08 ***
19:00–23:00 h-15:00–19:00 h -0.050024758 -0.092804244 -0.007245273  0.0220 *  
23:00–3:00 h-15:00–19:00 h  -0.012749310 -0.055528796  0.030030175  0.5586    
3:00–7:00 h-15:00–19:00 h    0.139636700  0.096857215  0.182416185 2.9e-10 ***
7:00–11:00 h-15:00–19:00 h   0.124943242  0.082163

RRuntimeError: Error in ezANOVA_main(data = data, dv = dv, wid = wid, within = within,  : 
  "rest" is not a variable in the data frame provided.
